## 从服务器上scp数据下来处理

### scp -r hhq@119.78.210.26:/home/hhq/mcmc11 /home/astrohhq/learngit/newwisedata/testssh_ans


In [86]:
import emcee
import corner
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pandas as pd
hvlist = np.loadtxt('hvlist.txt')
f = open(f'namelist.txt', "r", encoding="utf-8")
str1 = f.read()
namelist = str1.split()
mcmcdir = 'mcmc17'
print(namelist)

['1994CB', '1996GQ', '2002LT24', '2003MA3', '2008NP3', '2009UX17', '2010AJ30', '2010BH115', '2010CA55', '2010CC55', '2010CN141', '2010CO1', '2010DG77', '2010DH77', '2010EX11', '2010FA81', '2010FC81', '2010FH81', '2010GP67', '2010GS7', '2010HW81', '2010HX107', '2010HZ104', '2010JN71', '2010JX16', '2010KP10', '2010KU7', '2010KX7', '2010LH14', '2010LJ68', '2010LK68', '2010LL68', '2010LV108', '2010NS36', '2010NY65', '2010OH126', '2010PU66', '2010PW58', '2010XP69', '2014FD7', '2015TF', '2016UH101', '2019DD2']


In [87]:
for i in range(len(namelist)):
    if hvlist[i]>50:
        print(namelist[i])
'''
2010BH115 16.5
2010JX16 16.4
2010NS36 15.4
'''

2010BH115
2010JX16
2010NS36


'\n2010BH115 16.5\n2010JX16 16.4\n2010NS36 15.4\n'

建立一个 带空格的列表

In [88]:
name_space = []
for i in namelist:
    name_space.append(i[:4]+' '+i[4:])

读取wise的数据

In [89]:
df0 = pd.read_excel('mcwise.xlsx')
df0[:5]

,name,Dwise,Derr,albedo,Hv,P,other
0,1994 CB,193.0,10.0,0.2,NaN,8.676,NaN
1,1996 GQ,208.0,3.0,0.2,NaN,NaN,NaN
2,2002 LT24,143.0,24.0,0.2,NaN,NaN,NaN
3,2003 MA3,86.0,2.0,0.2,NaN,NaN,NaN
4,2008 NP3,193.0,3.0,0.2,23.3,NaN,NaN


生成当前目录计算结果的 列表

In [90]:
anslist = []
noans = []
#print('No ans :')
for i in range(len(namelist)):
    name = namelist[i]
    Hv = hvlist[i]
    if not os.path.exists(f'./{mcmcdir}/ansfit/fit.txt.{name}'):
        #print(name_space[i])
        noans.append(name)
        continue
    ans = np.loadtxt(f'./{mcmcdir}/ansfit/fit.txt.{name}')
    nn = len(np.loadtxt(f'./{mcmcdir}/datjd/mba.jd.{name}'))
    eta,eta1,eta2 = ans[0]
    D,D1,D2 = ans[1]
    wf,wf1,wf2 = ans[2]
    pv = ans[3][1]
    #print(name)
    anslist.append([name_space[i],nn,eta,wf,pv,D,D+D1,D-D2])
noans = np.array(noans)
#np.savetxt(f'{mcmcdir}_nan.txt',noans,fmt='%s')
#anslist

转换当前list为dataframe

In [91]:
df = pd.DataFrame(anslist,columns=['name','n','eta','wf','pv','D','Dup','Ddown'])
df = df.round({'eta':2,'D':1,'wf':2,'pv':2,'Dup':1,'Ddown':1})
#df.to_excel(f'{mcmcdir}.xlsx',index=False)
df[:5]

,name,n,eta,wf,pv,D,Dup,Ddown
0,1994 CB,21,3.14,0.0,0.28,132.5,132.5,132.5
1,1996 GQ,55,2.62,0.0,0.05,140.7,140.7,140.7
2,2002 LT24,45,2.35,0.0,0.25,102.8,102.8,102.7
3,2008 NP3,8,3.14,0.0,0.09,121.7,121.8,121.7
4,2009 UX17,130,1.75,0.5,0.17,162.3,162.3,162.3


合并wise的结果和计算结果

In [92]:
df.insert(loc=len(df.columns),column='D_wise',value=0)
df.insert(loc=len(df.columns),column='D_wiseErr',value=0)
df.insert(loc=4,column='pv_wise',value=0)
df[:5]

,name,n,eta,wf,pv_wise,pv,D,Dup,Ddown,D_wise,D_wiseErr
0,1994 CB,21,3.14,0.0,0,0.28,132.5,132.5,132.5,0,0
1,1996 GQ,55,2.62,0.0,0,0.05,140.7,140.7,140.7,0,0
2,2002 LT24,45,2.35,0.0,0,0.25,102.8,102.8,102.7,0,0
3,2008 NP3,8,3.14,0.0,0,0.09,121.7,121.8,121.7,0,0
4,2009 UX17,130,1.75,0.5,0,0.17,162.3,162.3,162.3,0,0


In [93]:
for i in range(len(df)):
    D  =df0[df0['name'] == df.loc[i,'name']]['Dwise']
    df.loc[i,'D_wise'] = float(D)
    Derr  =df0[df0['name'] == df.loc[i,'name']]['Derr']
    df.loc[i,'D_wiseErr'] = float(Derr)
    pv  =df0[df0['name'] == df.loc[i,'name']]['albedo']
    df.loc[i,'pv_wise'] = float(pv)

In [94]:
D_df = df[['name','D','D_wise','D_wiseErr']]
D_df.to_excel(f'./ans_excel/{mcmcdir}_D.xlsx')

In [95]:
df.to_excel(f'./ans_excel/{mcmcdir}.xlsx')